In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
from pathlib import Path
BASE_DIR = Path().absolute().parent.parent
sys.path.append(str(BASE_DIR))

In [3]:
from bayes_opt_adaptive  import BayesianOptimization
from bayes_opt_adaptive import UtilityFunction
from scipy.optimize import minimize, NonlinearConstraint, Bounds, SR1
from bayes_opt_adaptive.logger import JSONLogger
from bayes_opt_adaptive.event import Events
from bayes_opt_adaptive.util import load_logs
from cases import Case1,Case2,Case3
import numpy as np
import pandas as pd

In [ ]:
DIR = BASE_DIR/'scripts_res'/'Algorithms'
DIR

WindowsPath('d:/OneDrive/Current/2_Codes/BO_Related/ASBO/ASBO/scripts_res')

In [ ]:
** # Don't run all

### ASBO

In [5]:
re_run_ASBO = True
re_run_save_ASBO = True
sol = 'ASBO'
case_name = 'Case2'
runs_number = 100

In [17]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []
all_res_list = []

def func(x1,x2):
    x = np.array([x1,x2])
    obj, c1, = Case2(x)
    cons = [c1]
    return -obj,c1

nc_lb = np.array([-1e6])
nc_ub = np.array([0,])
cons = NonlinearConstraint(func, nc_lb, nc_ub)
pbounds = {'x1': (-5, 10),'x2': (0, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=2.5, xi=0, kappa_decay=1, kappa_decay_delay=0)

if re_run_ASBO == True or re_run_save_ASBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=100, n_iter=50,p_hyper=[[5, 2, 0.78, 0.95,1e-5]],acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint']])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        })
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)
        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+50))])

        if re_run_save_ASBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)

{'target': 268.6430578187194, 'params': {'x1': 3.247533685062293, 'x2': 0.0652332966212359}, 'constraint': -0.017350971511890023}
{'target': 268.6489479762007, 'params': {'x1': 3.2706788920182683, 'x2': 0.05459316706727202}, 'constraint': -0.019803675265670506}
{'target': 268.6338841913747, 'params': {'x1': 3.241863008162683, 'x2': 0.0681056596647866}, 'constraint': -0.0169978137702822}
{'target': 268.6474921520931, 'params': {'x1': 3.255067050007794, 'x2': 0.061681096850204044}, 'constraint': -0.018406015474599968}
{'target': 268.6540207016272, 'params': {'x1': 3.255352399334486, 'x2': 0.061333746131681195}, 'constraint': -0.017528886538865862}
{'target': 268.6374090719724, 'params': {'x1': 3.2545046014948102, 'x2': 0.0622725657241371}, 'constraint': -0.019735979073240983}
{'target': 268.64904619819055, 'params': {'x1': 3.264179967442051, 'x2': 0.05751778695448554}, 'constraint': -0.019431473031739088}
{'target': 268.6406415878431, 'params': {'x1': 3.2519879798890083, 'x2': 0.06330105

In [6]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1
count,100.000000,100.000000,100.000000,100.000000
mean,268.639351,3.252881,0.062946,-0.018452
std,0.038445,0.011534,0.006082,0.004095
min,268.486013,3.218602,0.051527,-0.038260
25%,268.635170,3.246834,0.058863,-0.019489
50%,268.647701,3.252768,0.062700,-0.017488
75%,268.660548,3.261400,0.065610,-0.015906
max,268.689022,3.277205,0.083002,-0.011993


### CBO

In [7]:
re_run_CBO = True
re_run_save_CBO = True
sol = 'CBO'
case_name = 'Case2'
runs_number = 100

In [20]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

obj_list = []
x_1_list = []
x_2_list = []
con_1_list  = []
all_res_list = []

def func_obj(x1,x2):
    x = np.array([x1,x2])
    obj, c1 = Case2(x)
    cons = [c1]
    return -obj

def func_cons(x1,x2):
    x = np.array([x1,x2])
    obj, c1 = Case2(x)
    return c1

nc_lb = np.array([-1e6])
nc_ub = np.array([0,])
cons = NonlinearConstraint(func_cons, nc_lb, nc_ub)
pbounds = {'x1': (-5, 10),'x2': (0, 15)}
acquisition_function = UtilityFunction(kind="ucb", kappa=1)

if re_run_CBO == True or re_run_save_CBO == True:
    for i in range(runs_number):
        optimizer = BayesianOptimization(f=func_obj, pbounds=pbounds,constraint=cons,verbose=0,random_state=i)
        optimizer.maximize(init_points=100, n_iter=50,acquisition_function=acquisition_function)
        print(optimizer.max)
        obj_list.extend([optimizer.max['target']])
        x_1_list.extend([optimizer.max['params']['x1']])
        x_2_list.extend([optimizer.max['params']['x2']])
        con_1_list.extend([optimizer.max['constraint']])
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        })
        all_res = [item['target'] for item in optimizer.res]
        all_fea = [item['allowed'] for item in optimizer.res]
        modified_res = [res if fea else -np.inf for res, fea in zip(all_res, all_fea)]
        all_res_list.append(modified_res)
        df2 = pd.DataFrame(all_res_list, columns=[list(range(100+50))])
                
        if re_run_save_CBO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
            df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)

{'target': 254.51117417656474, 'params': {'x1': 3.797694022151248, 'x2': 0.3016131928124033}, 'constraint': -0.3061983505082946}
{'target': 263.0013363474833, 'params': {'x1': 3.3957547308121363, 'x2': 0.1883397023867378}, 'constraint': -0.6958078432812682}
{'target': 252.83314556821014, 'params': {'x1': 3.740273327935519, 'x2': 0.3832641277453913}, 'constraint': -0.7678254093955195}
{'target': 240.59523336815826, 'params': {'x1': 2.710525897261629, 'x2': 1.30844053171207}, 'constraint': -1.9634887955332916}
{'target': 230.4210693623198, 'params': {'x1': 2.8254418366743703, 'x2': 1.6229007433084601}, 'constraint': -3.2952743975783445}
{'target': 199.1611327290972, 'params': {'x1': -4.102675448680481, 'x2': 14.474949604054984}, 'constraint': -0.44600181382647186}
{'target': 219.28025657052206, 'params': {'x1': 3.501951569875768, 'x2': 1.693775021807196}, 'constraint': -3.8849011707134693}
{'target': 214.99967883576042, 'params': {'x1': 8.968090295335326, 'x2': 0.3734884132552202}, 'cons

In [8]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,con_1
count,100.000000,100.000000,100.000000,100.000000
mean,234.125145,3.634077,1.449126,-1.808928
std,20.019028,2.280538,2.207689,1.274065
min,181.884020,-4.102675,0.075665,-4.410900
25%,220.265330,2.973281,0.583498,-2.702446
50%,237.068357,3.341231,0.930134,-1.620665
75%,249.837062,3.792171,1.635638,-0.694522
max,267.184927,9.673450,14.474950,-0.030862


### SCBO

In [9]:
re_run_SCBO = True
re_run_save_SCBO = True
sol = 'SCBO'
case_name = 'Case2'
runs_number = 100

In [6]:
x_1_list = []
x_2_list = []
iterations = []
max_values = []
all_res_list = []

itera = 0
import gpytorch
import torch
import tr_bo
from torch.quasirandom import SobolEngine
from botorch.utils.transforms import unnormalize
from botorch.models.model_list_gp_regression import ModelListGP

def func(x):
    global itera
    obj, c1 = Case2(x)
    cons = [c1]
    itera=itera+1
    return -obj,c1

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.double
dim = 2
batch_size = 1
n_init = 100 
bo_iter = 50 
max_cholesky_size = float("inf")  # Always use Cholesky

lb = torch.tensor([-1e6, ], dtype=torch.float64).to(device)
ub =  torch.tensor([0, ], dtype=torch.float64).to(device)
bounds = torch.tensor([[-5, 0,], [10, 15]]).to(device)

def objective(x):
    res = func(x)
    target = res[0]
    c1 = res[1]
    return target,c1
def run_multiple_calculations(num_runs):
    for random_seed in range(num_runs):
        try:
            def eval_objective(x):
                return objective(unnormalize(x, bounds))
            # Initialize TuRBO state
            state = tr_bo.ScboState(dim, batch_size=batch_size)
            sobol = SobolEngine(dim, scramble=True, seed=1)

            # Get initial data
            # Must get initial values for both objective and constraints
            train_X = tr_bo.get_initial_points(dim, n_init,seed=random_seed)

            train_Y_tensor_list = []
            C1_tensor_list = []

            for x in train_X:
                result = eval_objective(x)
                combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                train_Y, C1,= combined_tensor[0], combined_tensor[1]
                train_Y_tensor_list.append(train_Y)
                C1_tensor_list.append(C1)

            train_Y = torch.stack(train_Y_tensor_list, dim=0)
            C1 = torch.stack(C1_tensor_list, dim=0)

            while not state.restart_triggered:  # Run until TuRBO converges
                # Fit GP models for objective and constraints
                model = tr_bo.get_fitted_model(train_X, train_Y,dim)
                c1_model = tr_bo.get_fitted_model(train_X, C1,dim)

                # Generate a batch of candidates
                with gpytorch.settings.max_cholesky_size(max_cholesky_size):
                    X_next = tr_bo.generate_batch(
                        state=state,
                        model=model,
                        X=train_X,
                        Y=train_Y,
                        batch_size=batch_size,
                        n_candidates=2000,
                        constraint_model=ModelListGP(c1_model),
                        sobol=sobol,
                    )

                Y_next_tensor_list = []
                C1_next_tensor_list = []
                for x in X_next:
                    result = eval_objective(x)
                    combined_tensor = torch.tensor(result, dtype=dtype, device=device).unsqueeze(-1)
                    Y_next, C1_next,  = combined_tensor[0], combined_tensor[1]
                    Y_next_tensor_list.append(Y_next)
                    C1_next_tensor_list.append(C1_next)

                Y_next = torch.stack(Y_next_tensor_list, dim=0)
                C1_next = torch.stack(C1_next_tensor_list, dim=0)
                C_next = torch.cat([C1_next], dim=-1)
                state = tr_bo.update_state(state, Y_next, C_next,ub,lb)
                train_X = torch.cat((train_X, X_next), dim=0)
                train_Y = torch.cat((train_Y, Y_next), dim=0)
                C1 = torch.cat((C1, C1_next), dim=0)
                bool_tensor = (C1 >= lb.repeat(len(C1), 1)) & (C1 <= ub.repeat(len(C1), 1))
                bool_tensor = torch.all(bool_tensor, dim=-1)
                Valid_Y = train_Y[bool_tensor]
                Valid_X = train_X[bool_tensor]

                if len(Valid_Y)!=0:
                    max_index = torch.argmax(Valid_Y)  
                else:
                    print(f"{len(train_X)}) No feasible point yet! TR length: {state.length:.2e}")  

                if len(train_X) == n_init+ bo_iter:
                    modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
                    all_res_list.append(modified_Y.tolist())
                    df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])

                    if len(Valid_Y)!=0:                    
                        max_value = Valid_Y[max_index].item()
                        c_x = Valid_X[max_index]
                        print("obj= ",max_value, ' x = ', c_x)
                        max_values.append(max_value)
                        x_1_list.append(c_x[0].item())
                        x_2_list.append(c_x[1].item())
                    else:
                        print("No feasible solutions")
                        max_values.append(None)
                        x_1_list.append(None)
                        x_2_list.append(None)
                    iterations.append(len(train_X))
                    df = pd.DataFrame({
                    'obj': max_values,
                    'x_1': x_1_list,
                    'x_2': x_2_list,
                    'iterations': iterations
                    })
                    if re_run_save_SCBO == True:
                        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)     
                        df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)     
                    break          

        except Exception as e:
            fixed_length_Y = torch.full((n_init+bo_iter,), float('-inf'))           
            modified_Y = torch.where(bool_tensor, train_Y.flatten(), torch.tensor(float('-inf')))
            fixed_length_Y[:modified_Y.size(0)] = modified_Y
            all_res_list.append(fixed_length_Y.tolist())
            df2 = pd.DataFrame(all_res_list, columns=[list(range(n_init+bo_iter))])

            if len(train_X) == n_init + bo_iter:
                print(e)
            else:
                print(e, "Early Stop")
                if len(Valid_Y)!=0:
                    max_value = Valid_Y[max_index].item()
                    c_x = Valid_X[max_index]
                    print("obj= ",max_value, ' x = ', c_x)
                    max_values.append(max_value)
                    x_1_list.append(c_x[0].item())
                    x_2_list.append(c_x[1].item())
                else:   
                    print("obj= ", None, ' x = ', None)         
                    max_values.append(None)
                    x_1_list.append(None)
                    x_2_list.append(None)
            iterations.append(len(train_X))
            df = pd.DataFrame({
            'obj': max_values,
            'x_1': x_1_list,
            'x_2': x_2_list,
            'iterations': iterations
            })
            if re_run_save_SCBO == True:
                df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
                df2.to_csv(DIR/f'{sol}_Results2_{case_name}_{runs_number}.csv', index=False)
                
run_multiple_calculations(num_runs=runs_number)

All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  248.52925343003398  x =  tensor([0.5788, 0.0371], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  239.24424190054623  x =  tensor([0.5126, 0.0913], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  261.27405964828245  x =  tensor([0.5608, 0.0160], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  215.67780561063267  x =  tensor([0.5387, 0.1364], dtype=torch.float64)
All attempts to fit the model have failed. For more information, try enabling botorch.settings.debug mode. Early Stop
obj=  230.05095303304603  x =  tensor([0.4941, 0.1245], dtype=torch.float64)
All attempts to fit the m

In [10]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,100.000000,100.000000,100.000000,100.000000
mean,240.311136,0.544884,0.084330,109.280000
std,18.690925,0.088989,0.121552,5.740306
min,188.543562,0.085861,0.003501,102.000000
25%,229.973769,0.521838,0.033200,105.750000
50%,241.189705,0.548689,0.066886,107.000000
75%,255.366494,0.571639,0.095354,111.000000
max,268.021029,0.918790,0.951829,133.000000


In [ ]:
**

### trust-constr

In [11]:
re_run_tc = True
re_run_save_tc = True
sol = 'trust-constr'
case_name = 'Case2'
runs_number = 100

In [9]:
list_realvalue = []
initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1= Case2(x)
    cons = [c1]
    list_realvalue.extend([[obj]+cons])
    return obj,c1

def recyc(x):
    nc_lb = np.array([-1e6])
    nc_ub = np.array([0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-5,0] 
    ub = [10,15] 
    bd = Bounds(lb, ub, True) 
    try:
        result = minimize(objective, x0,  method='trust-constr', bounds=bd,constraints=cons)
        print('obj = ', result.fun, ', x = ', result.x)
        obj_list.append(result.fun)
        x_1_list.append(result.x[0])
        x_2_list.append(result.x[1])
        con_1_list.append(result.constr[0])
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        nfev_r.append(result.nfev)
        success_r.append(str(result.success))
        message_r.append(result.message)


    except Exception as e:
        print(e)
        obj_list.append(None)
        x_1_list.append(None)
        x_2_list.append(None)
        con_1_list.append(None)
        initial_x1.append(x[0][0])  
        initial_x2.append(x[0][1])  
        nfev_r.append(None)
        success_r.append(None)
        message_r.append(e)  
        

    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'con_1': con_1_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_tc == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    # return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_tc == True or re_run_save_tc == True:
    x_random = initial_random(lb=[-5,0],ub=[10,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)


obj =  -268.78838103316497 , x =  [3.27118983 0.04969917]
obj =  -200.66282779946073 , x =  [-4.16412556 14.79906521]
obj =  -268.7883810416733 , x =  [3.27118983 0.04969918]
obj =  -200.66282780022857 , x =  [-4.16412556 14.79906523]
obj =  -268.7883808282727 , x =  [3.27118982 0.04969919]
obj =  -268.7883810361163 , x =  [3.27118984 0.04969917]
obj =  -200.66282784363506 , x =  [-4.16412557 14.79906527]
obj =  -200.66282779605527 , x =  [-4.16412556 14.79906518]
`x0` violates bound constraints.
obj =  -220.17342701138534 , x =  [9.07220566 0.19079257]
obj =  -268.78838104174906 , x =  [3.27118984 0.04969917]
obj =  -268.78838080389164 , x =  [3.2711898 0.0496992]
obj =  -200.66282780023235 , x =  [-4.16412556 14.79906523]
obj =  -268.78838104166584 , x =  [3.27118983 0.04969918]
obj =  -200.66282779750193 , x =  [-4.16412556 14.7990652 ]
obj =  -268.7883810398107 , x =  [3.27118983 0.04969918]
obj =  -268.7883808061378 , x =  [3.2711898 0.0496992]
obj =  -268.7883808058441 , x =  [3.

In [12]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_cleaned = df.dropna(subset=['obj'])
df_cleaned.describe()

NaN count in column 'obj': 6


,obj,x_1,x_2,initial_x1,initial_x2,nfev
count,94.000000,94.000000,94.000000,94.000000,94.000000,94.000000
mean,-238.060729,1.322352,5.398575,2.307827,6.411201,191.521277
std,32.191274,4.551308,7.114508,4.034636,4.096552,424.135927
min,-268.788381,-4.164126,0.001972,-4.873275,0.195260,36.000000
25%,-268.788381,-4.164126,0.049699,-1.088702,2.849575,57.000000
50%,-268.788381,3.271190,0.049699,2.043980,6.332246,111.000000
75%,-200.662828,3.271190,14.799065,5.003062,9.402563,210.000000
max,-200.662828,9.072206,14.799065,9.907780,14.827646,3000.000000


### slsqp

In [13]:
re_run_slsqp = True
re_run_save_slsqp = True
sol = 'slsqp'
case_name = 'Case2'
runs_number = 100

In [12]:
initial_x1 = []
initial_x2 = []
x_1_list = []
x_2_list = []
obj_list = []
con_1_list  = []
con_2_list  = []
con_3_list  = []

nfev_r = []
success_r = []
message_r = []

def func(x):
    obj, c1= Case2(x)
    cons = [c1]
    return obj,c1

def recyc(x):
    nc_lb = np.array([-1e6])
    nc_ub = np.array([0])
    cons = NonlinearConstraint(lambda x: func(x)[1:4], nc_lb, nc_ub, hess=SR1(), finite_diff_rel_step=0.001)
    objective = lambda x: func(x)[0]
    x0=x[0]
    lb = [-5,0] 
    ub = [10,15] 
    bd = Bounds(lb, ub, True) 
    result = minimize(objective, x0,  method='slsqp', bounds=bd,constraints=cons)
    print('obj = ', result.fun, ', x = ', result.x)
    obj_list.append(result.fun)
    x_1_list.append(result.x[0])
    x_2_list.append(result.x[1])
 
    initial_x1.append(x[0][0])  
    initial_x2.append(x[0][1])  
    nfev_r.append(result.nfev)
    success_r.append(str(result.success))
    message_r.append(result.message)
    df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'initial_x1': initial_x1,
        'initial_x2': initial_x2, 
        'nfev': nfev_r,
        'success': success_r,
        'message': message_r,
        })    
    if re_run_save_slsqp == True:
        df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)
    return result.x

def initial_random(lb=[-10,-15],ub=[5,15],a=2,n_size=runs_number):
    bounds = np.array([lb,ub])
    bounds = bounds.T
    random_state = np.random.RandomState(a)
    x_random = random_state.uniform(bounds[:, 0], bounds[:, 1],
                                    size=(n_size, bounds.shape[0]))
    return x_random

if re_run_slsqp == True or re_run_save_slsqp == True:
    x_random = initial_random(lb=[-5,0],ub=[10,15])
    from itertools import product
    for combination in product(x_random):
        recyc(combination)

obj =  -268.7885046713122 , x =  [3.27302382 0.04886974]
obj =  -268.78850467678114 , x =  [3.27302381 0.04886974]
obj =  -268.7885048274905 , x =  [3.27302386 0.04886971]
obj =  -268.7885054032581 , x =  [3.27302381 0.04886972]
obj =  -268.78850480881397 , x =  [3.27302374 0.04886977]
obj =  -268.78850467530833 , x =  [3.27302377 0.04886976]
obj =  -200.66314216194593 , x =  [-4.16430535 14.811331  ]
obj =  -268.78851507991527 , x =  [3.27302379 0.0488694 ]
obj =  -268.7885048892059 , x =  [3.27302368 0.04886979]
obj =  -220.17428280328667 , x =  [9.07676871 0.19047851]
obj =  -268.7885047182179 , x =  [3.27302377 0.04886976]
obj =  -268.7885048738483 , x =  [3.27302377 0.04886975]
obj =  -200.66314217785478 , x =  [-4.16430573 14.8113588 ]
obj =  -268.78850466132735 , x =  [3.27302378 0.04886976]
obj =  -200.66314407376902 , x =  [-4.16430579 14.81135901]
obj =  -268.7885049436456 , x =  [3.27302243 0.04887035]
obj =  -268.78850467698624 , x =  [3.27302378 0.04886975]
obj =  -268.788

In [14]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['success'] == False).sum()
print(f"Count of rows where 'success' is False: {false_count}")
df_filtered = df[df['success'] != False]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'success' is False: 0


,obj,x_1,x_2,initial_x1,initial_x2,nfev
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,-253.612359,3.060658,1.986401,2.454588,6.561467,42.580000
std,26.213143,3.411885,4.982757,4.047803,4.187169,9.111187
min,-268.788522,-4.164306,0.048869,-4.873275,0.195260,22.000000
25%,-268.788505,3.273024,0.048870,-1.058449,2.930130,36.000000
50%,-268.788505,3.273024,0.048870,2.498894,6.449513,43.500000
75%,-220.174283,3.273024,0.190478,6.088579,9.812152,49.000000
max,-200.663142,9.076769,14.811364,9.907780,14.827646,70.000000


### PSO

In [15]:
re_run_PSO = True
re_run_save_PSO = True
sol = 'PSO'
case_name = 'Case2'
runs_number = 100

In [9]:
from pyswarm import pso

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0

def func(x):
    global itera
    obj, c1 = Case2(x)
    cons = [-c1]
    global x_p
    try:
        equal = np.array_equal(x,x_p)
        if not equal:
            itera=itera+1
    except:
        itera=itera+1
    x_p =x
    return obj,cons

lb = [-5,0] 
ub = [10,15] 
if re_run_PSO == True or re_run_save_PSO == True:
    for num in range(runs_number):
        xopt, fopt = pso(lambda x:func(x)[0], lb, ub,f_ieqcons=lambda x:func(x)[1:2], swarmsize=100, omega=0.5, phip=0.5, phig=0.5, maxiter=50, minstep=1e-8,minfunc=1e-8, debug=False)
        state = np.all(np.array(func(xopt)[1:])>= 0)
        itera=itera-1         
        print( 'fopt =', fopt, 'xopt = ', xopt, 'state = ', state) 
        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(fopt)
        x_1_list.append(xopt[0])
        x_2_list.append(xopt[1])
        state_list.append(state)
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_PSO == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)


Stopping search: maximum iterations reached --> 50
fopt = -268.787924398491 xopt =  [3.26968853 0.05039024] state =  True
Stopping search: maximum iterations reached --> 50
fopt = -268.7807160486949 xopt =  [3.25847118 0.05568653] state =  True
Stopping search: maximum iterations reached --> 50
fopt = -268.78846777062483 xopt =  [3.2739558  0.04845168] state =  True
Stopping search: maximum iterations reached --> 50
fopt = -268.7884345257296 xopt =  [3.27179844 0.04942348] state =  True
Stopping search: Swarm best objective change less than 1e-08
fopt = -268.7884019770098 xopt =  [3.27136989 0.04961743] state =  True
Stopping search: maximum iterations reached --> 50
fopt = -268.7824008937004 xopt =  [3.2843516  0.04398223] state =  True
Stopping search: maximum iterations reached --> 50
fopt = -268.7885001130914 xopt =  [3.27303932 0.04886291] state =  True
Stopping search: maximum iterations reached --> 50
fopt = -268.7848715100657 xopt =  [3.26314696 0.05343911] state =  True
Stoppi

In [16]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
false_count = (df['obj'] == 1e+100).sum()
print(f"Count of rows where 'obj' is 1e+100: {false_count}")
df_filtered = df[df['obj'] != 1e+100]
df_filtered = df_filtered[df_filtered['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0
Count of rows where 'obj' is 1e+100: 0


,obj,x_1,x_2,iterations
count,100.000000,100.000000,100.000000,100.000000
mean,-266.738665,3.044795,0.493927,5096.520000
std,11.679076,1.274214,2.528673,24.838257
min,-268.788504,-4.164413,0.043182,4884.000000
25%,-268.788402,3.263102,0.048441,5100.000000
50%,-268.787862,3.270971,0.049799,5100.000000
75%,-268.783554,3.273979,0.053460,5100.000000
max,-200.658671,3.286152,14.820000,5100.000000


### GA

In [17]:
re_run_GA = True
re_run_save_GA = True
sol = 'GA'
case_name = 'Case2'
runs_number = 100

In [6]:
from sko.GA import GA
from sko.operators import crossover

x_1_list = []
x_2_list = []
obj_list = []
iterations=[]
state_list = []
itera = 0

def func(x):
    global itera
    obj, c1= Case2(x)
    cons = [c1]
    itera=itera+1
    return obj,c1

lb = [-5,0] 
ub = [10,15] 
if re_run_GA == True or re_run_save_GA == True:
    for num in range(runs_number):
        ga = GA(func=lambda x:func(x)[0], n_dim=2, size_pop=100, max_iter=50, lb =[-5,0],ub=[10,15], precision=1e-7,prob_mut = 0.001, constraint_ueq=[lambda x:func(x)[1]])
        ga.register(operator_name='crossover', operator=crossover.crossover_2point_prob, crossover_prob=0.5)
        best_x, best_y = ga.run()
        state = np.all(np.array(func(best_x)[1:])<= 0)
        itera=itera-1 
        print( 'obj =', best_y, 'x = ', best_x,'state = ', state)

        if len(iterations)==0:
            iterations.append(itera)
        else:
            iterations.append(itera-sum(iterations))
        obj_list.append(best_y[0])
        x_1_list.append(best_x[0])
        x_2_list.append(best_x[1])
        state_list.append(state)
        df = pd.DataFrame({
        'obj': obj_list,
        'x_1': x_1_list,
        'x_2': x_2_list,
        'iterations': iterations,
        'state':state_list
        })
        if re_run_save_GA == True:
            df.to_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv', index=False)

obj = [-268.73786531] x =  [3.27183367 0.05109893] state =  True
obj = [-267.55276242] x =  [3.09339916 0.17260552] state =  True
obj = [-268.5626672] x =  [3.35037791 0.02184279] state =  True
obj = [-268.53220062] x =  [3.18947532 0.09533783] state =  True
obj = [-261.93462651] x =  [2.83602717 0.48751848] state =  True
obj = [-261.96018009] x =  [3.04938154 0.38326018] state =  True
obj = [-241.13015907] x =  [2.40891679 1.45357557] state =  True
obj = [-268.514623] x =  [3.35879016 0.01971446] state =  True
obj = [-250.02974091] x =  [2.55313101 1.05102573] state =  True
obj = [-257.15597777] x =  [2.7028641  0.72037166] state =  True
obj = [-262.04863493] x =  [2.83871916 0.48226286] state =  True
obj = [-265.75561794] x =  [2.98225417 0.28582787] state =  True
obj = [-264.96503969] x =  [2.94767278 0.32929039] state =  True
obj = [-268.54865853] x =  [3.35139559 0.02185866] state =  True
obj = [-253.01500128] x =  [2.63382734 0.90196817] state =  True
obj = [-268.32496691] x =  [

In [18]:
df = pd.read_csv(DIR/f'{sol}_Results_{case_name}_{runs_number}.csv')
nan_count = df['obj'].isna().sum()
print(f"NaN count in column 'obj': {nan_count}")
df_filtered = df[df['state'] == True]
df_filtered.describe()

NaN count in column 'obj': 0


,obj,x_1,x_2,iterations
count,100.000000,100.000000,100.000000,100.0
mean,-259.883767,2.877788,0.616409,10001.0
std,12.322445,0.745619,1.318011,0.0
min,-268.770959,-3.906796,0.016137,10001.0
25%,-267.201741,2.741899,0.174738,10001.0
50%,-263.680623,2.949682,0.370217,10001.0
75%,-257.611779,3.147579,0.691508,10001.0
max,-198.228007,4.116716,12.802492,10001.0
